### Example of using continuous integration to run data analysis

Russ Poldrack
March 13, 2022

In this simple example we will load data from [Eisenberg et al., 2019](https://github.com/IanEisenberg/Self_Regulation_Ontology) in which individuals were tested on a broad range of psychological measures as well as a survey of real-world outcomes.  We will ask the following question: Are scores on impulsivity measures associated with the likelihood of being arrested?

In [4]:
import os
from merge_data import load_data, merge_data
from arrest_analysis import model_arrest, load_alldata, \
     select_variables, save_output
import pandas as pd
import statsmodels.api as sm
from lrtest import lrtest

datadir = '../data'
resultsdir = '../results'

First we will load and clean up the data from the original study, which are stored in two files -- one containing the behavioral data, and the other containing the demographic data.

First we load and merge the two files into a single dataset, and save it for later use.

In [2]:
demogdata, taskdata = load_data(datadir)
alldata = merge_data(demogdata, taskdata, datadir)

print('Data merged and saved to data/alldata.csv')

Data merged and saved to data/alldata.csv


Next we will select the specific variables that we want.  We would like to include all variables from the following surveys:

- Barratt Impulsiveness Scale (BIS11)
- Sensation Seeking Survey
- Dickman survey
- Implusive Venturesomeness Survey
- UPPS-P

We also want to include a measure of prior arrest. The original dataset included a variable containing the number of times one had been arrested; however, this variable is zero-inflated with a long tail, so for the purposes of the present analysis we will binarize this variable to allow us to examine the likelihood of having been arrested at least once.  We also include Age and Sex as baseline variables.

In [3]:
arrestdata = select_variables(alldata)

arrestdata.columns

Index(['bis11_survey.Attentional', 'bis11_survey.Motor.logTr',
       'bis11_survey.Nonplanning', 'dickman_survey.functional',
       'impulsive_venture_survey.venturesomeness',
       'sensation_seeking_survey.boredom_susceptibility',
       'sensation_seeking_survey.disinhibition',
       'sensation_seeking_survey.experience_seeking',
       'sensation_seeking_survey.thrill_adventure_seeking',
       'upps_impulsivity_survey.lack_of_perseverance',
       'upps_impulsivity_survey.lack_of_premeditation',
       'upps_impulsivity_survey.negative_urgency',
       'upps_impulsivity_survey.positive_urgency',
       'upps_impulsivity_survey.sensation_seeking', 'Age', 'Sex',
       'EverArrested'],
      dtype='object')

Now let's fit our logistic regression model to the dataset.  

In [6]:
baseline_vars = ['Age', 'Sex']
y = arrestdata.loc[:, 'EverArrested']
X = sm.add_constant(arrestdata.drop(columns=['EverArrested']))
log_reg = sm.Logit(y, X).fit()
log_reg.summary()

Optimization terminated successfully.
         Current function value: 0.471928
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:           EverArrested   No. Observations:                  478
Model:                          Logit   Df Residuals:                      461
Method:                           MLE   Df Model:                           16
Date:                Sun, 13 Mar 2022   Pseudo R-squ.:                  0.1036
Time:                        08:08:28   Log-Likelihood:                -225.58
converged:                       True   LL-Null:                       -251.66
Covariance Type:            nonrobust   LLR p-value:                 1.034e-05
=====================================================================================================================
                                                        coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------
const                                                -5.3103      1.498     -3.544      0.000      -8.247      -2.373
bis11_survey.Attentional                              0.2050      0.159      1.292      0.196      -0.106       0.516
bis11_survey.Motor.logTr                              0.4566      0.928      0.492      0.623      -1.363       2.276
bis11_survey.Nonplanning                              0.1339      0.193      0.693      0.488      -0.245       0.512
dickman_survey.functional                            -0.4640      0.452     -1.027      0.304      -1.349       0.421
impulsive_venture_survey.venturesomeness              1.1095      1.068      1.039      0.299      -0.983       3.202
sensation_seeking_survey.boredom_susceptibility       0.3373      0.658      0.513      0.608      -0.952       1.627
sensation_seeking_survey.disinhibition                0.9320      0.587      1.587      0.112      -0.219       2.083
sensation_seeking_survey.experience_seeking          -0.7887      0.638     -1.236      0.216      -2.039       0.461
sensation_seeking_survey.thrill_adventure_seeking     0.3784      0.813      0.466      0.641      -1.214       1.971
upps_impulsivity_survey.lack_of_perseverance         -0.7480      0.325     -2.301      0.021      -1.385      -0.111
upps_impulsivity_survey.lack_of_premeditation         0.2214      0.385      0.575      0.566      -0.534       0.976
upps_impulsivity_survey.negative_urgency              0.5003      0.325      1.540      0.124      -0.137       1.137
upps_impulsivity_survey.positive_urgency             -0.2079      0.306     -0.680      0.496      -0.807       0.391
upps_impulsivity_survey.sensation_seeking            -0.0020      0.396     -0.005      0.996      -0.779       0.775
Age                                                   0.0455      0.016      2.894      0.004       0.015       0.076
Sex                                                  -0.6917      0.266     -2.602      0.009      -1.213      -0.171
=====================================================================================================================
"""

We see that this model fits the data fairly well, with a pseudo R-squared of about .10.  Let's compare this to a baseline model with only Age and Sex.

In [8]:
log_reg_baseline = sm.Logit(y, X[['const'] + baseline_vars]).fit()
log_reg_baseline.summary()

Optimization terminated successfully.
         Current function value: 0.510229
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:           EverArrested   No. Observations:                  478
Model:                          Logit   Df Residuals:                      475
Method:                           MLE   Df Model:                            2
Date:                Sun, 13 Mar 2022   Pseudo R-squ.:                 0.03088
Time:                        08:09:49   Log-Likelihood:                -243.89
converged:                       True   LL-Null:                       -251.66
Covariance Type:            nonrobust   LLR p-value:                 0.0004222
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.0951      0.494     -4.237      0.000      -3.064      -1.126
Age            0.0345      0.014      2.451      0.014       0.007       0.062
Sex           -0.7776      0.232     -3.354      0.001      -1.232      -0.323
==============================================================================
"""

This seems to fit the data substantially less well. We can do model comparison using a likelihood ratio test:

In [11]:
result = lrtest(log_reg, log_reg_baseline)
print('Likelihood ratio test results for full model vs baseline:')
print(f'Chi-squared: {result[0]}, p = {result[1]}')


Likelihood ratio test results for full model vs baseline:
Chi-squared: 36.61593762358501, p = 0.0008431973954732763


Based on this analysis, we conclude that impulsivity is related to the likelihood of being arrested.  